In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/dvl/Mask_RCNN

/content/gdrive/My Drive/dvl/Mask_RCNN


In [ ]:
#!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
ls

build/             LICENSE              output/           setup.cfg
DEFAULT_LOGS_DIR/  MANIFEST.in          README.md         setup.py
dist/              mask_rcnn.egg-info/  requirements.txt
images/            mrcnn/               samples/


In [ ]:
#! python setup.py install

In [ ]:
# check versions of tf and keras 
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
#!pip install keras==2.1.6
import keras
keras.__version__

TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


'2.1.6'

In [ ]:
import os
import sys
import time
import numpy as np
import imgaug  # https://github.com/aleju/imgaug (pip3 install imageaug)
import matplotlib
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
import zipfile
import urllib.request
import shutil

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Results directory
RESULTS_DIR = os.path.join(ROOT_DIR, "results")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
############################################################
#  Configurations
############################################################

class BasilConfig(Config):
    """Configuration for training on the nucleus segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "basil"

    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + leaf

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 75 #  train_length // batch_size
    VALIDATION_STEPS = 26 # max( val_length // batch_size)

    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between nucleus and BG
    DETECTION_MIN_CONFIDENCE = 0.5

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet101" ## change for different models

    # Select one entity (e.g. bounding boxes) out of many overlapping entities
    DETECTION_NMS_THRESHOLD = 0.3       

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 200  ## change for models (range 200 - 300)

    # Image mean (RGB)
    #MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 200  ##  change for models (range 200 - 300) 

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 40  ##  change for models (30) 

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 60  ##  change for models (range 200 - 300) 


class BasilInferenceConfig(BasilConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize imager for inferencing
   # IMAGE_RESIZE_MODE = "pad64"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7


In [ ]:
############################################################
#  Dataset
############################################################

class BasilDataset(utils.Dataset):

    def load_basil(self, dataset_dir, subset):
        """Load a subset of the COCO dataset.
        dataset_dir: The root directory of the COCO dataset.
        subset: What to load (train, val, test)
        class_ids: If provided, only loads images that have the given classes.
        """
        # load json with annotated images in COCO format
        coco = COCO("../segments2/kailkuhn_playground/v08.json")

        all_image_ids = list(coco.imgs.keys())

        # handle sets of train and val
        image_ids = []

        if subset == "train": 
                for image in all_image_ids:
                    if image <= len(all_image_ids)*.70:
                        image_ids.append(image)

        elif subset == "val":
            for image in all_image_ids:
                if image > len(all_image_ids)*.70 and image <= len(all_image_ids)*.95:
                    image_ids.append(image)

        elif subset == "test": # 11 images
            for image in all_image_ids:
                if image > len(all_image_ids)*.95:
                    image_ids.append(image)

        
        # Load all classes or a subset?
        class_ids = sorted(coco.getCatIds())

        # Add the class
        self.add_class("basil", 1, 'leaf')

        # Add images
        for i in image_ids:
           # print(i)
            self.add_image(
                "basil", image_id=i,
                path=os.path.join("../segments2/kailkuhn_playground/v.08", coco.imgs[i]['file_name']),
                width=coco.imgs[i]["width"],
                height=coco.imgs[i]["height"],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))



    def load_mask(self, image_id):
        """Load instance masks for the given image.

        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a BASIL image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "basil":
            print('wrong source')

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
       
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "basil.{}".format(annotation['category_id'])) 
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            print('calling return of parent')
            return super(BasilDataset, self).load_mask(image_id)


    # def image_reference(self, image_id):
    #     """Return a link to the image in the COCO Website."""
    #     info = self.image_info[image_id]
    #     if info["source"] == "basil":
    #         return "http://cocodataset.org/#explore?id={}".format(info["id"])
    #     else:
    #         super(BasilDataset, self).image_reference(image_id)

    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m


In [ ]:
############################################################
#  Training
############################################################

def train(model, dataset_dir, subset):
    """Train the model."""
    # Training dataset.
    dataset_train = BasilDataset()
    dataset_train.load_basil(dataset_dir, subset)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = BasilDataset()
    dataset_val.load_basil(dataset_dir, "val")
    dataset_val.prepare()

    # # Image augmentation
    # # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    # augmentation = iaa.SomeOf((0, 2), [
    #     iaa.Fliplr(0.5),
    #     iaa.Flipud(0.5),
    #     iaa.OneOf([iaa.Affine(rotate=90),
    #                iaa.Affine(rotate=180),
    #                iaa.Affine(rotate=270)]),
    #     iaa.Multiply((0.8, 1.5)),
    #     iaa.GaussianBlur(sigma=(0.0, 5.0))
    # ])

    print("Train network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=20, 
                # augmentation=augmentation,
                layers='heads')

## ALL LAYERS

    # print("Train all layers")
    # model.train(dataset_train, dataset_val,
    #             learning_rate=config.LEARNING_RATE,
    #             epochs=40,
    #             # augmentation=augmentation,
    #             layers='all')

In [ ]:
############################################################
#  RLE Encoding
############################################################

def rle_encode(mask):
    """Encodes a mask in Run Length Encoding (RLE).
    Returns a string of space-separated values.
    """
    assert mask.ndim == 2, "Mask must be of shape [Height, Width]"
    # Flatten it column wise
    m = mask.T.flatten()
    # Compute gradient. Equals 1 or -1 at transition points
    g = np.diff(np.concatenate([[0], m, [0]]), n=1)
    # 1-based indicies of transition points (where gradient != 0)
    rle = np.where(g != 0)[0].reshape([-1, 2]) + 1
    # Convert second index in each pair to lenth
    rle[:, 1] = rle[:, 1] - rle[:, 0]
    return " ".join(map(str, rle.flatten()))


def rle_decode(rle, shape):
    """Decodes an RLE encoded list of space separated
    numbers and returns a binary mask."""
    rle = list(map(int, rle.split()))
    rle = np.array(rle, dtype=np.int32).reshape([-1, 2])
    rle[:, 1] += rle[:, 0]
    rle -= 1
    mask = np.zeros([shape[0] * shape[1]], np.bool)
    for s, e in rle:
        assert 0 <= s < mask.shape[0]
        assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {}".format(shape, s, e)
        mask[s:e] = 1
    # Reshape and transpose
    mask = mask.reshape([shape[1], shape[0]]).T
    return mask


def mask_to_rle(image_id, mask, scores):
    "Encodes instance masks to submission format."
    assert mask.ndim == 3, "Mask must be [H, W, count]"
    # If mask is empty, return line with image ID only
    if mask.shape[-1] == 0:
        return "{},".format(image_id)
    # Remove mask overlaps
    # Multiply each instance mask by its score order
    # then take the maximum across the last dimension
    order = np.argsort(scores)[::-1] + 1  # 1-based descending
    mask = np.max(mask * np.reshape(order, [1, 1, -1]), -1)
    # Loop over instance masks
    lines = []
    for o in order:
        m = np.where(mask == o, 1, 0)
        # Skip if empty
        if m.sum() == 0.0:
            continue
        rle = rle_encode(m)
        lines.append("{}, {}".format(image_id, rle))
    return "\n".join(lines)


In [ ]:
############################################################
#  TRAIN START
############################################################

# Configurations
config = BasilConfig()
config.display()

# Create model
model = modellib.MaskRCNN(mode="training", config=config, model_dir= 'DEFAULT_LOGS_DIR')


# Select weights file to load
weights_path = COCO_MODEL_PATH

#weights_path = model.find_last()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        60
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [ ]:
# Load weights
print("Loading weights ", weights_path) 

# load call
model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])# Exclude the last layers because they require a matching number of classes



Loading weights  /content/gdrive/My Drive/dvl/mask_rcnn_coco.h5








In [ ]:
# Train CALL
train(model, os.path.join(ROOT_DIR, 'segments2/kailkuhn_playground/v.08'), 'train')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Train network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: DEFAULT_LOGS_DIR/basil20210104T1801/mask_rcnn_basil_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)


/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/20
74/75 [============================>.] - ETA: 1s - loss: 2.3535 - rpn_class_loss: 0.1085 - rpn_bbox_loss: 0.8480 - mrcnn_class_loss: 0.4413 - mrcnn_bbox_loss: 0.5269 - mrcnn_mask_loss: 0.4287

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2348: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


75/75 [==============================] - 187s 2s/step - loss: 2.3404 - rpn_class_loss: 0.1076 - rpn_bbox_loss: 0.8425 - mrcnn_class_loss: 0.4395 - mrcnn_bbox_loss: 0.5243 - mrcnn_mask_loss: 0.4265 - val_loss: 1.4243 - val_rpn_class_loss: 0.0363 - val_rpn_bbox_loss: 0.4871 - val_mrcnn_class_loss: 0.3420 - val_mrcnn_bbox_loss: 0.3133 - val_mrcnn_mask_loss: 0.2457

Epoch 2/20
75/75 [==============================] - 124s 2s/step - loss: 1.3601 - rpn_class_loss: 0.0481 - rpn_bbox_loss: 0.4964 - mrcnn_class_loss: 0.3183 - mrcnn_bbox_loss: 0.2748 - mrcnn_mask_loss: 0.2226 - val_loss: 1.3131 - val_rpn_class_loss: 0.0363 - val_rpn_bbox_loss: 0.4318 - val_mrcnn_class_loss: 0.3443 - val_mrcnn_bbox_loss: 0.2753 - val_mrcnn_mask_loss: 0.2254
Epoch 3/20
75/75 [==============================] - 124s 2s/step - loss: 1.1853 - rpn_class_loss: 0.0393 - rpn_bbox_loss: 0.4209 - mrcnn_class_loss: 0.2883 - mrcnn_bbox_loss: 0.2344 - mrcnn_mask_loss: 0.2025 - val_loss: 1.2136 - val_rpn_class_loss: 0.0331 - va